In [1]:
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')
    print(os.getcwd())

/scratch/iav225/ccm/project/DQN-Atari-Transfer-Learning


In [3]:
import torch
import gym
import numpy as np

In [4]:
import utils
from agent import Agent
from estimator import Estimator
from tqdm.notebook import tqdm

In [ ]:
name = 'adam_batchnorm_kaiming-01'
env_id = 'AssaultNoFrameskip-v4'

### No-clip env

In [5]:
env = utils.wrap_deepmind(utils.make_atari(env_id, max_episode_steps=None, frameskip=4), clip_rewards=False, frame_stack=True, stacks=4)

In [6]:
estimator = Estimator(env.action_space.n)

In [7]:
estimator.load_state_dict(torch.load(f'logs/{name}/model.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [8]:
def play_episode(env, estimator):
    state = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        with torch.no_grad():
            action = np.argmax(estimator(torch.tensor(np.array(state).astype(np.float32) / 255.0).unsqueeze(0)).cpu().numpy())
        
        state, reward, done, info = env.step(action)
        total_reward += reward
        
    return total_reward

In [13]:
scores = []

In [14]:
for i in tqdm(range(100)):
    scores.append(play_episode(env, estimator))

In [20]:
print(f'Mean score: {np.mean(scores):.3f} +- {np.std(scores):.3f}')

Mean score: 621.600 +- 551.774
